# Steps for building/testing MOM6 for GPU offload

This is following [the report from Marshall](https://github.com/marshallward/mom6-gpu-report/blob/main/gpu_report.rst#testing-in-mom6) which documents the compilation steps and testing. This notebook documents the process for Gadi.

In [23]:
cd /scratch/tm70/$USER
git clone --recursive https://github.com/NOAA-GFDL/MOM6-examples.git

Cloning into 'MOM6-examples'...
remote: Enumerating objects: 72067, done.        
remote: Counting objects: 100% (12919/12919), done.        
remote: Compressing objects: 100% (853/853), done.        
remote: Total 72067 (delta 12511), reused 12091 (delta 12066), pack-reused 59148 (from 2)        
Receiving objects: 100% (72067/72067), 70.38 MiB | 20.51 MiB/s, done.
Resolving deltas: 100% (62551/62551), done.
Updating files: 100% (3278/3278), done.
Submodule 'src/FMS1' (https://github.com/NOAA-GFDL/FMS) registered for path 'src/FMS1'
Submodule 'src/FMS2' (https://github.com/NOAA-GFDL/FMS) registered for path 'src/FMS2'
Submodule 'src/MOM6' (https://github.com/NOAA-GFDL/MOM6.git) registered for path 'src/MOM6'
Submodule 'src/SIS2' (https://github.com/NOAA-GFDL/SIS2.git) registered for path 'src/SIS2'
Submodule 'src/atmos_null' (https://github.com/NOAA-GFDL/atmos_null.git) registered for path 'src/atmos_null'
Submodule 'src/coupler' (https://github.com/NOAA-GFDL/coupler.git) registered f

In [24]:
module load intel-compiler/2021.10.0 netcdf/4.7.4 python3-as-python openmpi/4.1.7
cd MOM6-examples/ocean_only
make -j 4

mkdir -p build
make -C ../shared/fms \
  BUILD=/scratch/tm70/ey7514/MOM6-examples/shared/fms/build \
  CODEBASE=/scratch/tm70/ey7514/MOM6-examples/src/FMS2
cp ../src/MOM6/ac/configure.ac build/configure.ac
cp -r ../src/MOM6/ac/m4 build
cp ../src/MOM6/ac/Makefile.in build/Makefile.in
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
mkdir -p /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cp ../../src/MOM6/ac/deps/Makefile.fms.in /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/Makefile.in
cp ../../src/MOM6/ac/deps/configure.fms.ac /scratch/tm70/ey7514/MOM6-examples/shared/fms/build/configure.ac
cp -r ../../src/MOM6/ac/deps/m4 /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
autoreconf build
autoreconf /scratch/tm70/ey7514/MOM6-examples/shared/fms/build
cd /scratch/tm70/ey7514/MOM6-examples/shared/fms/build && \
  PATH="/apps/openmpi/wrapper/fortran:/apps/openmpi/wrapper:/apps/openmpi/4.1.7/bin:/apps/netcdf/4.7.4/bin:/apps/intel-ct/wrapper:/apps/intel

In [25]:
cd double_gyre
mpiexec -n 1 ../build/MOM6

NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =   44
  Y-AXIS =   40
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =   22
  Y-AXIS =   20
NOTE: diag_manager_mod::diag_manager_init: diag_manager is using fms2_io
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_init is called with time_init present.
 MOM_in domain decompos

Things ran ok, let's profile for hotspots.

In [26]:
module load intel-vtune

In [27]:
rm -rf mom6-prof-vtune*
mpiexec -n 1 vtune -collect hotspots -r mom6-prof-vtune ../build/MOM6

vtune: Analyzing data in the node-wide mode. The hostname (gadi-login-07.gadi.nci.org.au) will be added to the result path/name.
vtune: Warning: Only user space will be profiled due to credentials lack. Consider changing /proc/sys/kernel/perf_event_paranoid file for enabling kernel space profiling.
vtune: Collection started.
NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =   44
  Y-AXIS =   40
 MOM_inc domai

In [29]:
vtune -report=hotspots -r mom6-prof-vtune.gadi-login-07.gadi.nci.org.au -format=csv | column -ts $'\t'

vtune: Using result path `/scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-prof-vtune.gadi-login-07.gadi.nci.org.au'
vtune: Executing actions 100 % done                                            
Function                              CPU Time  CPU Time:Effective Time  CPU Time:Spin Time  CPU Time:Overhead Time  Module                Function (Full)                       Source File                 Start Address
btstep                                1.133479  1.133479                 0.0                 0.0                     MOM6                  btstep                                MOM_barotropic.F90          0x61b3f0
vertvisc_coef                         0.323425  0.323425                 0.0                 0.0                     MOM6                  vertvisc_coef                         MOM_vert_friction.F90       0x10af790
meridional_flux_adjust                0.242178  0.242178                 0.0                 0.0                     MOM6                  me

In [30]:
vtune -report=hotspots -r mom6-prof-vtune.gadi-login-07.gadi.nci.org.au -format=csv -group-by=source-file-path | column -ts $'\t'

vtune: Using result path `/scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-prof-vtune.gadi-login-07.gadi.nci.org.au'
vtune: Executing actions 100 % done                                            
Source File Path                                                                                                        CPU Time  CPU Time:Effective Time  CPU Time:Spin Time  CPU Time:Overhead Time
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_barotropic.F90                                                 1.406373  1.406373                 0.0                 0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_continuity_PPM.F90                                             1.197303  1.197303                 0.0                 0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/parameterizations/vertical/MOM_vert_friction.F90                        0.689873  0.689873                 0.0                 0.0
[Unknown]                                             

I'm not confident that I'm using VTune correctly, but it looks like MOM_barotropic is an expensive step.

But looks like `core/MOM_barotropic.F90:btstep` is the most time-consuming function.

Turns out it's a 2.5k line subroutine: https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_barotropic.F90#L2848

Otherwise some of the subroutines in `core/MOM_continuity_PPM.F90` like [`zonal_flux_adjust`](https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_continuity_PPM.F90#L1094) or [`meridional_flux_adjust`](https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_continuity_PPM.F90#L1995) look like more manageable candidates.

Let's see if the `benchmark` test case looks any different.

In [31]:
cd ../benchmark

In [32]:
rm -rf mom6-prof-vtune*
mpiexec -n 1 ../build/MOM6

NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to   955296.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =  360
  Y-AXIS =  180
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =  180
  Y-AXIS =   90
NOTE: diag_manager_mod::diag_manager_init: diag_manager is using fms2_io
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_

: 139

Weird... Maybe it's a problem with the intel compiler? Let's try build again...

I spent an afternoong looking at this with no luck.

Mighang told me it's because of the OOM killer on the login nodes, so I needed to run the test in a job. I'm used to getting a more informative message from cgroups, but idk how it's setup on Gadi...

Anyway, looking at the results of the vtune report:

In [1]:
# job submission script:
cd /scratch/tm70/$USER/MOM6-examples/ocean_only/benchmark
echo '#!/bin/bash
#PBS -N MOM6_test
#PBS -P tm70
#PBS -q normal
#PBS -l walltime=02:00:00
#PBS -l ncpus=48
#PBS -l mem=192GB
#PBS -l jobfs=10GB
#PBS -l storage=scratch/tm70
#PBS -l wd

module purge
module load netcdf intel-compiler openmpi intel-vtune

MOM6_EXEC=../build/MOM6

mpirun -np $PBS_NCPUS vtune -collect hotspots -r mom6-prof-vtune $MOM6_EXEC > mom6.out 2> mom6.err' > submit.sh

qsub submit.sh

133140840.gadi-pbs


In [2]:
# per subroutine
# generated with 
# vtune -report=hotspots -r mom6-prof-vtune.gadi-cpu-clx-1883.gadi.nci.org.au/ -format=csv > prof.csv
head prof.csv | column -ts $'\t'

Function                         CPU Time   CPU Time:Effective Time  CPU Time:Spin Time  CPU Time:Overhead Time  Module       Function (Full)                  Source File                 Start Address
btstep                           54.262846  54.262846                0.0                 0.0                     MOM6         btstep                           MOM_barotropic.F90          0x61b3f0
uct_mm_iface_progress            49.399108  49.399108                0.0                 0.0                     libuct.so.0  uct_mm_iface_progress            mm_iface.c                  0x18260
uct_mm_iface_poll_fifo           43.640615  43.640615                0.0                 0.0                     libuct.so.0  uct_mm_iface_poll_fifo           mm_iface.c                  0x18262
ucp_worker_progress              42.150527  42.150527                0.0                 0.0                     libucp.so.0  ucp_worker_progress              ucp_worker.c                0x48370
vertvisc_coef     

In [3]:
# grouped by source
# generated with
# vtune -report=hotspots -r mom6-prof-vtune.gadi-cpu-clx-1883.gadi.nci.org.au/ -format=csv  -group-by=source-file-path > prof-grouped.csv
head prof-grouped.csv | column -ts $'\t'

Source File Path                                                                                               CPU Time    CPU Time:Effective Time  CPU Time:Spin Time  CPU Time:Overhead Time
/jobfs/126523667.gadi-pbs/0/ucx/1.17.0/build/src/uct/../../../source/ucx-1.17.0/src/uct/sm/mm/base/mm_iface.c  145.993677  145.993677               0.0                 0.0
[Unknown]                                                                                                      108.672162  104.853139               3.819023            0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_continuity_PPM.F90                                    77.840111   77.840111                0.0                 0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_barotropic.F90                                        62.977219   62.977219                0.0                 0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/parameterizations/vertical/MOM_vert_friction.F90               54.716628 

Ignoring the UCX and `[Unknown]`, looks like the outcome is roughly the same where `MOM_baratropic.F90:btstep` takes the longest, followed by smaller subroutines in `MOM_continuity_PPM.F90`.